# How to use this code
## !Use Google Colab:
https://colab.research.google.com/drive/10wDa-BbR8-PtjY-IDImhhqRnixb8UJU6
1. Run chunk 1
2. Define the correct file path for the training data in Chunk 2 + run chunk 2
3. Run chunk 3

### To perform neste resampling:
Adapt params in chunk 4 and run it
This stores the best model as a pth file which can be ignored
### To train model given parameters and save model as .pkl:
Adapt paramt in chunk 5 as explained and then run it. The .pkl file is requiered for getting predictions on test data.
### To get model predictions on test data:
In chunk 6 define the path to the .pkl model file, as well as to expression and pData test data sets. Then run the chunk.

### To get survival data and curves:
- perform steps 1-3
-  In chunk 7: define the pkl file and training data (same as used for the pkl file)
- run the chunk 7
- in chunk 8: define the model the model stored through chunk 7, as well as the expression data for which survival should be predicted. Dim have to be the same as for the training

In [ ]:
### Chunk 1
# Installing and laoding packages
!pip install lifelines
!pip install scikit-learn==1.5.2
!pip install scikit-survival==0.23.1
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import torch
from lifelines.utils import concordance_index
from sklearn.utils.validation import check_X_y, check_is_fitted
import logging
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils import check_random_state
from sksurv.util import Surv
import os
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict
import copy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=fd97718f521d0cd6fb9e06edc566d85e9a46f0ffefbb93a5076371cf6f5043ca
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 22.7 MB/s eta 0:00:00


In [ ]:
### Chunk 2
# Defining the pathways to the data used for model training.
# One expression data file and one pData file is needed.
# As for standard input, common genes and intersect genes are used. One is commented out.
# /content is the folder which serves as the standard upload folder in google colab
#EXPRESSION_DATA_PATH = '/content/exprs_intersect.csv'
EXPRESSION_DATA_PATH = '/content/common_genes_knn_imputed.csv'
CLINICAL_DATA_PATH = '/content/merged_imputed_pData.csv'

In [ ]:
# Chunk 3

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class DeepSurvNet(nn.Module):
    """
    PyTorch based neural network architecture designed for survival prediction.
    This network consists of fully connected layers with ReLU activation,
    dropout for regularization, and a final layer that outputs a single
    hazard prediction value.
    """

    def __init__(self, n_features, hidden_layers=[32, 16], dropout=0.2):
        super().__init__()
        layers = []
        prev_size = n_features
        self.model = None

        # Build hidden layers
        for size in hidden_layers:
            layers.extend([
                nn.Linear(prev_size, size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_size = size

        # Output layer (1 node for hazard prediction)
        layers.append(nn.Linear(prev_size, 1, bias=False))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class DeepSurvModel(BaseEstimator, RegressorMixin):
    """
    Implementation of the DeepSurv model that integrates
    with scikit-learn, specifying  configurable architecture,
    training procedures, and evaluation metrics.

    The model includes:
    - Customizable neural network architecture
    - Mini-batch training with early stopping
    - CPU/GPU support
    - Concordance index evaluation
    - Compatibility with scikit-learn's cross-validation and pipeline features
    - Reproducible training through seed control

    The model follows scikit-learn's estimator interface by implementing
    fit(), predict(), get_params() and set_params() methods.
    """
    def __init__(self, n_features=None, hidden_layers=[256,128], dropout=0.4,
                 learning_rate=0.0001, device='cpu', random_state=123,
                 batch_size=64, num_epochs=500, patience=10):
        # Initialize hyperparameters and device settings
        self.n_features = n_features
        self.hidden_layers = hidden_layers
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.device = device if torch.cuda.is_available() and device == 'cuda' else 'cpu'
        self.random_state = random_state
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.patience = patience
        # Set random seeds for reproducibility
        torch.manual_seed(random_state)
        np.random.seed(random_state)

        self.scaler = StandardScaler()
        self.model = None
        self.is_fitted_ = False
        self.training_history_ = {'train_loss': [], 'val_loss': []}
        self.n_features_in_ = None

    def fit(self, X, y):
        # Validate input data
        X, y = check_X_y(X, y, accept_sparse=True)

        # Set random seeds at the beginning of training
        np.random.seed(self.random_state)
        torch.manual_seed(self.random_state)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(self.random_state)
            torch.cuda.manual_seed_all(self.random_state)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        # Initialize model and data loader
        self.n_features_in_ = X.shape[1]
        self.init_network(self.n_features_in_)
        self.model.to(self.device)

        train_loader, val_loader = self._prepare_data(X, y, val_split=0.1)

        best_val_loss = float('inf')
        best_model_state = None
        counter = 0.0
        # Training loop
        for epoch in range(self.num_epochs):
            self.model.train()
            epoch_loss_ = 0.0
            n_batches_ = 0
            for X_batch, time_batch, event_batch in train_loader:
                loss = self._train_step(X_batch, time_batch, event_batch)
                epoch_loss_ += loss
                n_batches_ += 1
            avg_train_loss = epoch_loss_ / n_batches_
            self.training_history_['train_loss'].append(avg_train_loss)

            # Validation
            self.model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for X_batch, time_batch, event_batch in val_loader:
                    val_loss += self._eval_step(X_batch, time_batch, event_batch)

            val_loss = val_loss / len(val_loader)
            self.training_history_['val_loss'].append(val_loss)

            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = copy.deepcopy(self.model.state_dict())
                counter = 0
            else:
                counter += 1

            if counter > self.patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Restore best model
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)

        self.is_fitted_ = True
        return self

    def predict(self, X):
        # Predict risk scores for new data
        check_is_fitted(self, 'is_fitted_')
        if isinstance(X, pd.DataFrame):
            X = X.values
        X = torch.FloatTensor(X).to(self.device)
        self.model.eval()
        with torch.no_grad():
            risk_scores = self.model(X).cpu().numpy()
        return risk_scores.flatten()

    def score(self, X, y):
        # Calculate concordance index
        check_is_fitted(self, 'is_fitted_')
        preds = self.predict(X)
        return self.c_index(-preds, y)

    def get_params(self, deep=True):
        # Return model parameters
        return {
            "n_features": self.n_features,
            "hidden_layers": self.hidden_layers,
            "dropout": self.dropout,
            "learning_rate": self.learning_rate,
            "device": self.device,
            "random_state": self.random_state,
            "batch_size": self.batch_size,
            "num_epochs": self.num_epochs,
            "patience": self.patience
        }

    def set_params(self, **parameters):
        # Set model parameters
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def clone(self):
        super(self).clone()

    def _prepare_data(self, X, y, val_split = 0.1):
      # Split data into training and validation sets
      X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_split, random_state=self.random_state)

      X_scaled_train = X_train
      times_train = np.ascontiguousarray(y_train['time']).astype(np.float32)
      event_field_train = 'status' if 'status' in y_train.dtype.names else 'event'
      events_train = np.ascontiguousarray(y_train[event_field_train]).astype(np.float32)
      X_tensor_train = torch.FloatTensor(X_scaled_train).to(self.device)
      time_tensor_train = torch.FloatTensor(times_train).to(self.device)
      event_tensor_train = torch.FloatTensor(events_train).to(self.device)

      X_scaled_val = X_val
      times_val = np.ascontiguousarray(y_val['time']).astype(np.float32)
      event_field_val = 'status' if 'status' in y_val.dtype.names else 'event'
      events_val = np.ascontiguousarray(y_val[event_field_val]).astype(np.float32)
      X_tensor_val = torch.FloatTensor(X_scaled_val).to(self.device)
      time_tensor_val = torch.FloatTensor(times_val).to(self.device)
      event_tensor_val = torch.FloatTensor(events_val).to(self.device)

      # Create DataLoader with reproducible generator
      train_dataset = TensorDataset(X_tensor_train, time_tensor_train, event_tensor_train)
      val_dataset = TensorDataset(X_tensor_val, time_tensor_val, event_tensor_val)

      generator = torch.Generator()
      generator.manual_seed(self.random_state)

      train_loader = DataLoader(
          train_dataset,
          batch_size=self.batch_size,
          shuffle=True,
          generator=generator
      )

      val_loader = DataLoader(
          val_dataset,
          batch_size=self.batch_size,
          shuffle=True,
          generator=generator
      )

      return train_loader, val_loader

    def _negative_log_likelihood(self, risk_pred, times, events):
        # Calculate negative log-likelihood loss
        _, idx = torch.sort(times, descending=True)
        risk_pred = risk_pred[idx]
        events = events[idx]
        log_risk = risk_pred
        risk = torch.exp(log_risk)
        cumsum_risk = torch.cumsum(risk, dim=0)
        log_cumsum_risk = torch.log(cumsum_risk + 1e-10)
        event_loss = events * (log_risk - log_cumsum_risk)
        return -torch.mean(event_loss)

    def _train_step(self, X, times, events):
        # Perform one training step
        self.optimizer.zero_grad()
        risk_pred = self.model(X)
        loss = self._negative_log_likelihood(risk_pred, times, events)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def _eval_step(self, X, times, events):
        # Evaluate model on validation data
        risk_pred = self.model(X)
        loss = self._negative_log_likelihood(risk_pred, times, events)
        return loss.item()

    def _check_early_stopping(self, counter):
        if len(self.training_history_['val_loss']) < 2:
            return 0.0

        if self.training_history_['val_loss'][-1] < self.training_history_['val_loss'][-2]:
            counter = 0.0
        else:
            counter += 1.0
        return counter

    def c_index(self, risk_pred, y):
        # Calculate concordance index
        if not isinstance(y, np.ndarray):
            y = y.detach().cpu().numpy()
        event_field = 'status' if 'status' in y.dtype.names else 'event'
        time = y['time']
        event = y[event_field]
        if not isinstance(risk_pred, np.ndarray):
            risk_pred = risk_pred.detach().cpu().numpy()
        if np.isnan(risk_pred).all():
            return np.nan
        return concordance_index(time, risk_pred, event)

    def init_network(self, n_features):
        # Initialize the neural network and optimizer
        self.model = DeepSurvNet(n_features=n_features, hidden_layers=self.hidden_layers, dropout=self.dropout).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)


def _get_survival_subset(y, indices):
    """Extract survival data subset while preserving structure"""
    subset = np.empty(len(indices), dtype=y.dtype)
    event_field = 'status' if 'status' in y.dtype.names else 'event'
    subset[event_field] = y[event_field][indices]
    subset['time'] = y['time'][indices]
    return subset



def _aggregate_results(results):
    """Aggregates nested CV results."""
    scores = [res['test_score'] for res in results]
    if np.isnan(scores).all():
        logger.warning(f"Found only NaN values in CV-results: {scores}")
        mean_score, std_score = np.nan, np.nan
    else:
        mean_score = np.nanmean(scores)
        std_score = np.nanstd(scores)

    logger.info(f"Aggregated results:")
    logger.info(f"Mean score: {mean_score:.3f} ± {std_score:.3f}")
    logger.info(f"Individual scores: {scores}")

    return {
        'mean_score': mean_score,
        'std_score': std_score,
        'fold_results': results
    }

def nested_resampling(estimator, X, y, groups, param_grid, monitor = None, ss = GridSearchCV,
                     outer_cv = LeaveOneGroupOut(), inner_cv = LeaveOneGroupOut(), scoring = None):
    """Implementation of the nested resampling logic for hyperparameter optimization"""

    logger.info("Starting nested resampling...")
    logger.info(f"Data shape: X={X.shape}, groups={len(np.unique(groups))} unique")

    outer_results = []

    # Create reproducible splits
    splits = list(outer_cv.split(X, y, groups))

    for i, (train_idx, test_idx) in enumerate(splits):
        logger.info(f"\nOuter fold {i+1}")

        # Set seeds for this fold
        fold_seed = 42 + i
        np.random.seed(fold_seed)
        torch.manual_seed(fold_seed)

        X_train = X.iloc[train_idx]
        X_test = X.iloc[test_idx]
        y_train = _get_survival_subset(y, train_idx)
        y_test = _get_survival_subset(y, test_idx)
        train_groups = groups[train_idx] if groups is not None else None

        test_cohort = groups[test_idx][0] if groups is not None else None
        logger.info(f"Test cohort: {test_cohort}")

        # Perform inner cross-validation using grid search
        inner_gcv = ss(estimator, param_grid, cv = inner_cv, refit = True, n_jobs=1, verbose = 2)
        if monitor is not None:
            inner_results = inner_gcv.fit(X_train, y_train, groups = train_groups, model__monitor = monitor)
        else:
            inner_results = inner_gcv.fit(X_train, y_train, groups = train_groups)

        # Retrieve results from inner CV
        inner_cv_results = inner_results.cv_results_
        inner_best_params = inner_results.best_params_

        # Evaluate the best model on the outer test set
        outer_model = inner_results.best_estimator_.named_steps['model']
        test_score = outer_model.score(X_test, y_test)

        logger.info(f"Best parameters: {inner_best_params}")
        logger.info(f"Test score: {test_score:.3f}")

        # Append results for this outer fold
        outer_results.append({
            'test_cohort': test_cohort,
            'test_score': test_score,
            'best_params': inner_best_params,
            'inner_cv_results': inner_cv_results
        })

    return _aggregate_results(outer_results)



class ModellingProcess():
    """
    This class manages the entire modeling process including data preparation,
    nested cross-validation, model training, and result saving. It is a
    standardized way of modeling used for several of the implemented mode types
    and supports both simple training and complex nested resampling approaches.
    Results can be automatically saved and evaluated.
    """
    def __init__(self) -> None:
        self.outer_cv = LeaveOneGroupOut()
        self.inner_cv = LeaveOneGroupOut()
        self.ss = GridSearchCV
        self.pipe = None
        self.cmplt_model = None
        self.cmplt_pipeline = None
        self.nrs = None
        self.X = None
        self.y = None
        self.groups = None
        self.path = None
        self.fname_cv = None

    def prepare_survival_data(self, pdata):
        # Convert survival data into structured format
        status = pdata['BCR_STATUS'].astype(bool).values
        time = pdata['MONTH_TO_BCR'].astype(float).values
        y = Surv.from_arrays(
            event=status,
            time=time,
            name_event='status',
            name_time='time'
        )
        return y

    def prepare_data(self):
        # Load the feature matrix (X) and survival data (y)
        # If this is used ouside of google colab the pathways need to be ajusted
        self.X = pd.read_csv(EXPRESSION_DATA_PATH, index_col=0)
        self.y = self.prepare_survival_data(pd.read_csv(CLINICAL_DATA_PATH, index_col=0))
        self.groups = np.array([idx.split('.')[0] for idx in self.X.index])

    def do_modelling(self, pipeline_steps, config):
        # Set random seed for reproducibility
        self._set_seed()

        # Set parameters for the modelling process if provided in config
        if config.get("params_mp", None) is not None:
            self.set_params(config['params_mp'])

        # Set file paths for saving results if provided
        if config.get("path", None) is None or config.get("fname_cv", None) is None:
            logger.warning("Didn't get sufficient path info for saving cv-results")
        else:
            self.path = config['path']
            self.fname_cv = config['fname_cv']

        # Check modelling prerequisites
        err, mes = self._check_modelling_prerequs(pipeline_steps)
        if err:
            logger.error("Requirements setup error: %s", mes)
            raise Exception(mes)
        else:
            self.pipe = Pipeline(pipeline_steps)

        # Extract configuration values for nested resampling
        param_grid, monitor, do_nested_resampling, refit_hp_tuning = self._get_config_vals(config)

        try:
            logger.info("Start model training...")
            logger.info(f"Input data shape: X={self.X.shape}")

            # Perform nested resampling if enabled
            if do_nested_resampling:
                logger.info("Nested resampling...")
                self.nrs = nested_resampling(
                    self.pipe, self.X, self.y, self.groups, param_grid, monitor, self.ss, self.outer_cv, self.inner_cv
                )
                # Save nested resampling results if paths are provided
                if (self.fname_cv is not None) and (self.path is not None):
                    self.save_results(self.path, self.fname_cv, model=None, cv_results=self.nrs, pipe=None)
        except Exception as e:
            logger.error(f"Error during nested resampling: {str(e)}")
            raise

        # Perform hyperparameter tuning if enabled
        if refit_hp_tuning:
            try:
                logger.info("Do HP Tuning for complete model; refit + set complete model")
                self.fit_cmplt_model(param_grid)
                # Save the complete model if paths are provided
                if (self.fname_cv is not None) and (self.path is not None):
                    self.save_results(self.path, self.fname_cv, model=self.cmplt_model, cv_results=None, pipe=self.cmplt_pipeline)
            except Exception as e:
                logger.error(f"Error during complete model training: {str(e)}")
                raise
        elif not refit_hp_tuning and not do_nested_resampling:
            # Fit the entire pipeline without hyperparameter tuning
            logger.info("Fit complete pipeline without HP tuning (on default params)")
            self.cmplt_pipeline = self.pipe.fit(self.X, self.y)
            # Save the pipeline if paths are provided
            if (self.fname_cv is not None) and (self.path is not None):
                self.save_results(self.path, self.fname_cv, model=None, cv_results=None, pipe=self.cmplt_pipeline)

        return self.nrs, self.cmplt_model, self.cmplt_pipeline

    def fit_cmplt_model(self, param_grid, monitor=None):
        # Perform hyperparameter tuning on the full dataset
        logger.info("Do HP Tuning for complete model")
        res = self.ss(
            estimator=self.pipe,
            param_grid=param_grid,
            cv=self.outer_cv,
            n_jobs=1,
            verbose=2,
            refit=True
        )
        if monitor is not None:
            res.fit(self.X, self.y, groups=self.groups, model__monitor=monitor)
        else:
            res.fit(self.X, self.y, groups=self.groups)
        self.resampling_cmplt = res
        self.cmplt_pipeline = res.best_estimator_
        self.cmplt_model = res.best_estimator_.named_steps['model']
        return res.best_estimator_.named_steps['model'], res

    def save_results(self, path, fname, model=None, cv_results=None, pipe=None):
        # Save model and results
        if model is None:
            logger.warning("Won't save any model, since it's not provided")
        else:
            # Save the model
            model_dir = os.path.join(path, 'model')
            os.makedirs(model_dir, exist_ok=True)
            model.model.to(torch.device('cpu'))
            torch.save(model.model, os.path.join(model_dir, f"{fname}.pth"))
            logger.info(f"Saved model to {model_dir}")

        if cv_results is None:
            logger.warning("Won't save any CV results, since it's not provided")
        else:
            # Save cross-validation results
            results_dir = os.path.join(path, 'results')
            os.makedirs(results_dir, exist_ok=True)
            results_file = os.path.join(results_dir, f"{fname}_cv.csv")
            pd.DataFrame(cv_results).to_csv(results_file)
            logger.info(f"Saved CV results to {results_file}")

    def _check_modelling_prerequs(self, pipeline_steps):
        # Check if data and pipeline are correctly set up
        err = False
        mes = ""
        if self.X is None or self.y is None:
            mes += "1) Please call prepare_data() with your preferred config or set X, y, and groups"
            err = True
        if not any('model' in tup for tup in pipeline_steps):
            mes += "2) Caution! Your pipeline must include a step named 'model' for the model"
            err = True
        return err, mes

    def _get_config_vals(self, config):
        # Extract configuration values from the config dictionary
        if config.get("params_cv", None) is None:
            logger.warning("No param grid for (nested) resampling detected - will fit model with default HPs on complete data")
            return None, False, False, False
        if config.get('monitor', None) is None:
            logger.info("No additional monitoring detected")
        return config['params_cv'], config.get('monitor', None), config.get('do_nested_resampling', True), config.get('refit', True)

    def set_params(self, params):
        # Set attributes based on the provided parameters
        for key, value in params.items():
            setattr(self, key, value)

    def _set_seed(self, seed=1234):
        # Set random seeds for reproducibility
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        global random_state
        random_state = check_random_state(seed)




In [ ]:
### Chunk 4
# Define hyperparameter grid for the nested resampling. To do so, params_cv can be adapted.
# refit and do_nested_resampling should be true
# fname_cv is the name by which the the results are stored in a csv file ->adapt
MODEL_CONFIG = {
    'params_cv'  : {
        # Define the number and size of hidden layers
        'model__hidden_layers': [[512, 256, 128, 64],[512, 256, 128],[512, 128],[512, 256], [256, 128], [1024], [512], [256], [128]],
        # Learning rate for optimization
        'model__learning_rate': [0.00001, 0.0001],
        # Batch size for training
        'model__batch_size': [64],
        # Number of training epochs
        'model__num_epochs': [500],
        # Dropout rate for regularization
        'model__dropout': [0.2, 0.4],
        'model__device': ['cuda']
    },
    'refit': True,
    'do_nested_resampling': True,
    'path' : "",
    'fname_cv' : 'example_name'
}


mp = ModellingProcess()
mp.prepare_data()

ds_pipeline_steps = [
    ('model', DeepSurvModel())
]

nstd_res_result, cmplt_model, cmplt_pipeline = mp.do_modelling(ds_pipeline_steps, MODEL_CONFIG)

print(nstd_res_result)

Fitting 8 folds for each of 1 candidates, totalling 8 fits
Early stopping at epoch 18
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   6.2s
Early stopping at epoch 14
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   0.8s
Early stopping at epoch 17
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.2s
Early stopping at epoch 17
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.2s
Early stopping at epoch 14
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001

Early stopping at epoch 17
Fitting 9 folds for each of 1 candidates, totalling 9 fits
Early stopping at epoch 12
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.3s
Early stopping at epoch 17
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   0.9s
Early stopping at epoch 12
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   0.9s
Early stopping at epoch 15
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.4s
Early stopping at epoch 21
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[128], 

Early stopping at epoch 15
{'mean_score': 0.6387820979811811, 'std_score': 0.08108262343902252, 'fold_results': [{'test_cohort': 'Atlanta_2014_Long', 'test_score': 0.6219653179190752, 'best_params': {'model__batch_size': 64, 'model__device': 'cuda', 'model__dropout': 0.2, 'model__hidden_layers': [128], 'model__learning_rate': 0.0001, 'model__num_epochs': 500}, 'inner_cv_results': {'mean_fit_time': array([1.68204659]), 'std_fit_time': array([1.70513945]), 'mean_score_time': array([0.00602192]), 'std_score_time': array([0.00320306]), 'param_model__batch_size': masked_array(data=[64],
             mask=[False],
       fill_value=999999), 'param_model__device': masked_array(data=['cuda'],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_model__dropout': masked_array(data=[0.2],
             mask=[False],
       fill_value=1e+20), 'param_model__hidden_layers': masked_array(data=[list([128])],
             mask=[False],
       fill_value='?',
            dt

In [ ]:
### Chunk 5
# Define parameters for training the DeepSurv model
model_params = {
     'hidden_layers': [256, 128],
     'learning_rate': 0.00001,
     'batch_size': 64,
     'num_epochs': 500,
     'dropout': 0.2,
     'device': 'cuda',
     'random_state': 123
 }
# Initialize the ModellingProcess class and load data

mp = ModellingProcess()
mp.prepare_data()

# Train the model on the full dataset
model_to_save = DeepSurvModel(**model_params)

# Make predictions on the training data
model_to_save.fit(mp.X.values, mp.y)

# save
preds_train = model_to_save.predict(mp.X.values)
save_dir = "/content/my_saved_model"
os.makedirs(save_dir, exist_ok=True)
model_file = os.path.join(save_dir, "deep_surv_model_common_genes.pkl")

with open(model_file, 'wb') as f:
     pickle.dump(model_to_save, f)

Early stopping at epoch 47


In [ ]:
### Chunk 6

# Load model
model_file = "/content/deep_surv_model_intersect[256, 128].pkl"
with open(model_file, 'rb') as f:
    loaded_model = pickle.load(f)


# Load Test Data
# Expression Data
X_test = pd.read_csv('/content/example_exprs.csv', index_col=0)

# Clinical Data containing survival data
test_pdata = pd.read_csv('/content/example_pData.csv', index_col=0)

# Prepare survival data
test_status = test_pdata['BCR_STATUS'].astype(bool).values
test_time = test_pdata['MONTH_TO_BCR'].astype(float).values
y_test = Surv.from_arrays(
    event=test_status,
    time=test_time,
    name_event='status',
    name_time='time'
)

# Predict on test data
test_predictions = loaded_model.predict(X_test.values)
print("\First 5:")
print(test_predictions[:5])

# Calculate c-index
test_cindex = loaded_model.c_index(-test_predictions, y_test)
print("\nC-index on test data:", test_cindex)

# Sore results
results_df = pd.DataFrame({
    'sample_id': X_test.index,
    'risk_score': test_predictions
})
results_df.to_csv('/content/test_predictions.csv')


\First 5:
[ 0.03913932  0.01209671 -0.0101697   0.29160768  0.16612417]

C-index on test data: 0.7681590601915999


In [ ]:
# Chunk 7
"""
This Code chunk adds the baseline hazard to a trained model. To do so the
model, as well as training data are loaded, then the breslow baseline hazard
is generated and finally the new model with baseline hazard is stored.
"""
# Load the pre-trained DeepSurv model
model_file = "/content/deep_surv_model_common_genes[256, 128].pkl"
with open(model_file, 'rb') as f:
    deep_surv_model = pickle.load(f)



# Load training data (features and survival information)
X_train = pd.read_csv("/content/common_genes_knn_imputed.csv", index_col=0)
pData_train = pd.read_csv("/content/merged_imputed_pData.csv", index_col=0)

# Extract survival time and event status
times_train = pData_train["MONTH_TO_BCR"].values.astype(float)
events_train = pData_train["BCR_STATUS"].astype(bool).values

# 3) Compute Breslow estimator for baseline hazard
def breslow_baseline_hazard(model, X, times, events):
    log_risk = model.predict(X)
    risk = np.exp(log_risk)

    # Ensure numerical stability for small values
    risk = np.clip(risk, 1e-10, None)

    # Sort data by survival times
    order = np.argsort(times)
    sorted_times = times[order]
    sorted_events = events[order]
    sorted_risk = risk[order]

    # Identify unique event times
    unique_event_times = np.unique(sorted_times[sorted_events == 1])

    bhaz = []
    at_risk_sum = np.zeros_like(unique_event_times)
    event_count = np.zeros_like(unique_event_times)

    # Vectorized computation of baseline hazard
    for i, t in enumerate(unique_event_times):
        at_risk = sorted_risk[sorted_times >= t]
        at_risk_sum[i] = at_risk.sum()
        event_count[i] = np.sum((sorted_times == t) & (sorted_events == 1))

    bhaz = event_count / np.maximum(at_risk_sum, 1e-8)
    cbhaz = np.cumsum(bhaz)

    return unique_event_times, bhaz, cbhaz

# Compute the baseline hazard using the Breslow method
unique_times, bhaz, cbhaz = breslow_baseline_hazard(
    model=deep_surv_model,
    X=X_train.values,
    times=times_train,
    events=events_train
)


# 4) Save the baseline hazard in the model object
deep_surv_model.unique_event_times_ = unique_times
deep_surv_model.cum_baseline_hazard_ = cbhaz

# Optionally save the model with the baseline hazard included
with open("/content/deep_surv_model_common_genes[256, 128]_inclBH.pkl", 'wb') as f:
    pickle.dump(deep_surv_model, f)





In [ ]:
# Chunk 8
"""This chunk predicts survival based on the prev. stored model"""
# Load the model with stored baseline hazard
with open("/content/deep_surv_model_common_genes[256, 128]_inclBH.pkl", 'rb') as f:
    deep_surv_model_with_bh = pickle.load(f)

# Load new test data
X_test = pd.read_csv("/content/intersect_genes_test_cohort1_low_risk.csv", index_col=0)


# Compute survival function for test patients
def predict_survival(model, X_new):
    """
    Erwartet, dass model.unique_event_times_ und model.cum_baseline_hazard_
    vorhanden sind.
    """
    # Compute risk scores
    log_risk = model.predict(X_new)
    risk = np.exp(log_risk)

    # Compute survival probabilities S(t) = exp(-Lambda_0(t) * risk)
    times = model.unique_event_times_
    cbhaz = model.cum_baseline_hazard_

    surv_list = []
    for lam_0_t in cbhaz:
        surv_list.append(np.exp(-lam_0_t * risk))


    surv = np.vstack(surv_list)
    return times, surv

# Generate survival curves for test patients
times_vec, surv_mat = predict_survival(deep_surv_model_with_bh, X_test.values)

print("Survival-Kurven berechnet!")
print(f"surv_mat.shape = {surv_mat.shape}, times_vec.shape = {times_vec.shape}")

# Plot survival curves (demo)
plt.figure(figsize=(7,5))
if X_test.shape[0] == 1:
    plt.step(times_vec, surv_mat[:,0], where="post", label="Patient")
else:
    for i, pat_id in enumerate(X_test.index):
        plt.step(times_vec, surv_mat[:, i], where="post", label=str(pat_id))



surv_vec = surv_mat[:, 0]
df_single = pd.DataFrame({
    'time': times_vec,
    'survival': surv_vec
})
df_single.to_csv("patient_survival_deep_surv_exprs_example.csv", index=False)


